In [ ]:
import numpy as np
import cv2
import imutils

verbose = True

image = cv2.imread('/mnt/estudo_tiago/upx_4_ai/bases_estudo/12 - Deteccao de placas e Pytesseract/2.jpg')

image = imutils.resize(image, width=500)

if verbose == True:
    cv2.imshow("Imagem original", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

if verbose == True:
    cv2.imshow("Imagem cinza", gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
gray = cv2.bilateralFilter(gray, 11,17,17)

if verbose == True:
    cv2.imshow("Imagem filtrada", gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

edged = cv2.Canny(gray, 170, 200)

if verbose == True:
    cv2.imshow("Imagem com canny", edged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
cnts, _, = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

img1 = image.copy()
cv2.drawContours(img1, cnts, -1, (0,255,0), 3)

if verbose == True:
    cv2.imshow("Imagem contornos", img1)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
cnts = sorted(cnts, key = cv2.contourArea, reverse=True)[:30]
NumPlacaCnt = None

img2 = image.copy()
cv2.drawContours(img2, cnts, -1, (0, 255, 0),3)

if verbose == True:
    cv2.imshow("Imagem TOP 30", img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
count = 0
idx = 1

for c in cnts:
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02*peri, True)
    if len(approx) == 4:
        NumPlacaCnt = approx
        
        x,y,w,h = cv2.boundingRect(c)
        new_img = gray[y:y+h, x:x+w]
        cv2.imwrite('Placa' + str(idx) + '.png', new_img)
        break
    
cv2.drawContours(image, [NumPlacaCnt], -1, (0,255,0), 3)

if verbose == True:
    cv2.imshow("Imagem final com placa detectada", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
Cropped_img_loc = 'Placa', str(idx), + '.png'

if verbose == True:
    cv2.imshow("Imagem cropped placa", cv2.imread(Cropped_img_loc))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

pytesseract.pytesseract.tesseract_cmd = r'/usr/share/doc/'



Face com blur para LGPD

In [1]:
import cv2
import os
    
cascade = cv2.CascadeClassifier('/mnt/estudo_tiago/upx_4_ai/bases_estudo/11 - Faces/Haarcascades/haarcascade_frontalface_default.xml')

def faceBlur(gray, frame):
    faces = cascade.detectMultiScale(gray, 1.1, 4)
    for(x,y,w,h) in faces:
        roi_faces = frame[y:y+h, x:x+w]
        blur = cv2.GaussianBlur(roi_faces, (101,101), 0)
        frame[y:y+h, x:x+w] = blur
    
    return frame

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = faceBlur(gray, frame)
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(10) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cap.release()    

Landmarks em faces humanas

In [ ]:
import cv2
import dlib

p = '/mnt/estudo_tiago/upx_4_ai/bases_estudo/11 - Faces/shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 0)
    
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        for j in range(1,68):
            cv2.putText(frame, str(j), (shape.part(j).x, shape.part(j).y), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 0.3, color=(0,0,255))
    
    cv2.imshow("Frame", frame)
    k = cv2.waitKey(10)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

Classificador de Haar (Detecção de Faces)

In [ ]:
import cv2

face_cascade = cv2.CascadeClassifier('./bases_estudo/11 - Faces/Haarcascades/haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    for (x, y, w, h) in faces:
        cv2. rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
    
    cv2.imshow('Faces detectadas', frame)
    
    k = cv2.waitKey(30)
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

Diferenciação de frames - Fluxo Ótico

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

mask = np.zeros_like(first_frame)

mask[..., 1] = 255

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, pyr_scale=0.5, levels=3, winsize=15, iterations=3, poly_n=5, poly_sigma=1.1, flags=0)
    
    #computar a magnitude e and do optical flow dos vetores 2D
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    #print(magnitude)
    #print(angle)
    
    #Setar o hue de acordo com o fluxo optico
    mask[..., 0] = angle * 180 /np.pi/2
    
    #setar a imagem de acordo com a magnitude de fluxo optico (normalizado)
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    
    #converter HSV para BGR
    rgb = cv2. cvtColor(mask, cv2.COLOR_HSV2BGR)
    
    #soma ponderada
    dense_flow = cv2.addWeighted(frame, 1, rgb, 2, 0)
    
    cv2.imshow("Dense optical flow", dense_flow)
    prev_gray = gray
    
    if cv2.waitKey(10) & 0xFF == ord ('q'):
        break

cap.release()
cv2.destroyAllWindows()

Diferenciação de frame com MOG2 e KNN

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

mog = cv2.createBackgroundSubtractorMOG2(history=300, varThreshold=10, detectShadows=True)
knn = cv2.createBackgroundSubtractorKNN(history=100, dist2Threshold=40, detectShadows=True)

while (True):
    ret, frame = cap.read()
    
    fgmask1 = mog.apply(frame)
    fgmask2 = knn.apply(frame)
    
    cv2.imshow('MOG2', fgmask1)
    cv2.imshow('KNN', fgmask2)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Frame Diff (Diferenciação de frames)

In [ ]:
import cv2

cap = cv2.VideoCapture(0)
ret, frame = cap.read()
prox_frame = frame

while True:
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    prox_frame_gray = cv2.cvtColor(prox_frame, cv2.COLOR_RGB2GRAY)
    
    frame_diff = cv2.absdiff(frame_gray, prox_frame_gray)
    
    cv2.imshow('frame diff', frame_diff)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    prox_frame = frame.copy()
    ret, frame = cap.read()
    
cap.release()
cv2.destroyAllWindows()

ROI - Área de interesse

In [ ]:
import cv2
import numpy as np

#imagem de teste
image = cv2.imread('teste_roi.png')
image_to_show = np.copy(image)

#etados iniciais do mouse
cropping = False
x_init, y_init, top_left_pt, bottom_right_pt = 0, 0, 0, 0

def roi(event, x, y, flags, param):
    global image_to_show, x_init, y_init, top_left_pt, bottom_right_pt, cropping
    
    if event == cv2.EVENT_LBUTTONDOWN:
        cropping = True
        x_init, y_init = x, y
        image_to_show = np.copy(image)
        print(f'Ponto inicial em X {x_init}')
        print(f'Ponto inicial em Y {y_init}')
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if cropping == True:
            image_to_show = np.copy(image)
            cv2.rectangle(image_to_show, (x_init, y_init),
                          (x, y), (0, 255, 0), 1)
            
    elif event == cv2.EVENT_LBUTTONUP:
        cropping = False
        top_left_pt, bottom_right_pt = x, y
        print(f'Ponto superior {top_left_pt}')
        print(f'Ponto inferior {bottom_right_pt}')
        
cv2.namedWindow('image')
cv2.setMouseCallback('image', roi)

while True:
    cv2.imshow('image', image_to_show)
    k = cv2.waitKey(1)

    if k == ord('c'):
        if y_init > bottom_right_pt:
            y_init, bottom_right_pt = bottom_right_pt, y_init
        if x_init > top_left_pt:
            x_init, top_left_pt = top_left_pt, x_init
            
        if bottom_right_pt - y_init > 1 and top_left_pt - x_init > 0:
            image = image[y_init:bottom_right_pt, x_init:top_left_pt]
            print(image)
            image_to_show = np.copy(image)

    if k == ord('s'):
        cv2.imwrite('teste.jpg', image_to_show)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

Retângulos nas imagens

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

#callback
def draw_rect(event, x, y, flags, params):
    global pt1, pt2, topLeftClicked, bottomRightClicked
    if event == cv2.EVENT_LBUTTONDOWN:
        if topLeftClicked == True and bottomRightClicked == True:
            pt1 = (0,0)
            pt2 = (0,0)
            topLeftClicked = False
            bottomRightClicked = False
        
        if topLeftClicked == False:
            pt1 = (x, y)
            topLeftClicked = True
        elif bottomRightClicked == False:
            pt2 = (x,y)
            bottomRightClicked = True
            

#variaveis globais
pt1 = (0,0)
pt2 = (0,0)
topLeftClicked = False
bottomRightClicked = False

cv2.namedWindow('teste')
cv2.setMouseCallback('teste', draw_rect)

while True:
    
    ret,frame = cap.read()
    
    # desenhar retangulo
    if topLeftClicked:
        cv2.circle(frame, center=pt1, radius=5, color=(0,0,255))
    if topLeftClicked and bottomRightClicked:
        cv2.rectangle(frame, pt1, pt2, color=(0,0,255), thickness=1)
    
    cv2.imshow('teste', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

""" width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#parte superior
x = width // 2
y = height // 2

#Altura 

w = width // 4
h = height // 4

#Exibir
 """
while True:
    
    ret,frame = cap.read()
    
    """ cv2.rectangle(frame, (x,y), (w+x, y+h), color=(0,0,255), thickness=4) """
    
    cv2.imshow('video', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows

Acessando meu vídeo

In [ ]:
import cv2
import time

cap = cv2.VideoCapture('/mnt/tiago-estudo/upx_4_ai/video1.mp4')

if cap.isOpened() == False:
    print("ERROR!")
    
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret == True:
        time.sleep(1/5)
        
        cv2.imshow('frame', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
    else:
        break
    
cap.release()
cv2.destroyAllWindows()
        

Acessar web cam

In [ ]:
import cv2

cap = cv2.VideoCapture(0)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret,frame = cap.read()
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()